<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/EntityResolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entity Resolution in ArangoDB**

This notebook will dive into the world of Entity Resolution in ArangoDB. 

This notebook is one of a few ways you can learn about Entity Resolution with ArangoDB:
* [Entity Resolution Lunch and Learn video](https://www.arangodb.com/resources/lunch-sessions/graph-beyond-lunch-break-15-entity-resolution/)
* It is the interactive version of the [Entity Resolution Blog Post](https://www.arangodb.com/2021/07/entity-resolution-in-arangodb/)
* There is a runnable example demo available on [ArangoDB Oasis](https://cloud.arangodb.com/) in the 'Examples' tab.


In this notebook we will:

*   give a brief background in Entity Resolution (ER)
*   discuss some use-cases for ER
*   discuss some techniques for performing ER in ArangoDB

# **What is Entity Resolution?**

Entity Resolution is the process of disambiguating records of real-world entities that are represented multiple times in a database or across multiple databases.

An entity is a unique thing (person, company, product, etc.) in the real world with a set of attributes that describes it (a name, zip/postal code, gender, deviceID, title, price, product category, etc.). The single entity might have multiple references across multiple data sources. For example, a single user might have two different email addresses, and a company might have multiple phone numbers in CRM and ERP systems. Many real-world datasets do not contain unique identifiers. In such cases, we have to use a combination of fields to identify unique entities across records by grouping or linking them together.
Entity Resolution (ER) is a process akin to data deduplication that aims to uniquely resolve data that potentially comes from multiple sources to a single real-world entity. The applications for entity resolution are wide and varied across industry verticals, including: 
 * fraud detection 
 * KYC
 * recommendations engine
 * customer 360

Entity Resolution is an ideal use-case for a graph database like ArangoDB. In subsequent sections, we will discuss the steps to take and things to consider when you build ER applications with ArangoDB.



# Setup

Before getting started with ArangoDB, we need to prepare our environment and create a database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!git clone -b entity_resolution --single-branch https://github.com/arangodb/interactive_tutorials.git entity_resolution
!rsync -av entity_resolution/ ./data --exclude=.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!chmod -R 755 ./tools

!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import oasis

from pyArango.connection import *
from arango import ArangoClient

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName='EntityResolution', credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
# Please note that we use the python-arango driver as it has better support for ArangoSearch 
database = oasis.connect_python_arango(login)

In [ ]:
print("https://"+login["hostname"]+":"+str(login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Feel free to use the above URL to checkout the ArangoDB WebUI!

# **Entity Resolution in ArangoDB - first, build a graph**

The first step is to create a graph representing the entities you wish to resolve and their corresponding attributes. Data can be read from a CSV file, perhaps ETL’ed from multiple data sources.

# **Import Sample Dataset**

For this notebook, we will import a sample dataset. The arangorestore command below will only work on Linux or Windows systems; if you want to run this notebook on a different OS, please consider using the appropriate arangorestore from the Download area.

In [ ]:
! ./tools/arangorestore -c none --create-collection true --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --include-system-collections --replication-factor 3  --input-directory "./data/entity_dump"

Once the graph has been populated, this is how the sample schema would look like





##### Figure 1
!["User_Attributes"](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/entity_detection_images/User_Attributes2-768x622.png?raw=1)



In this example, the entity is a user with attributes like first name, last name, ip address, email, phone and device ID.  These are represented as vertices and edges connecting the entity to its attributes.  As the graph gets populated, we will start to see attributes that are shared by multiple entities.

##### Figure 2

!["overlapping-attributes"](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/entity_detection_images/Overlapping_Attr2.png.png?raw=1)

The subgraph above shows multiple user entities with a number of shared attributes, as shown by the edges.  You can see User 309 and User 36 share a total of 5 attributes in common (email, first name, phone, ip address and last name) whereas User 308 only shares 2 attributes with User 36.

Modeling the data like this in a graph makes it very easy to look for matching attributes between users.  As we will see in the next section, matching attributes between users is key when you are looking for similarities between them.

### Computing similarity between entities

At the heart of any entity resolution task or algorithm is the computation of similarity.  

How similar is User 36 compared to User 309?  Which users amongst User 65, 309, 307, 306, 301, and 308 are most similar to User 36?  The ability to answer these questions will allow you to build a probabilistic graph that can be used for more graph analytics downstream.

One can utilize the typical similarity algorithms like Jaccard or Cosine similarity in ArangoDB. Or perhaps you need something custom that is domain specific.  All these can be achieved and expressed in AQL (ArangoDB Query Language).

### Example 1 – Find Similar Users using Jaccard in AQL

In [ ]:
results = database.aql.execute("""
// For each input_user attribute
FOR attr, edge, p IN 1..1 OUTBOUND 'Users/36' GRAPH 'IDGraph'

     // Get all users with shared attributes (like FirstName, LastName,..., Phone, DeviceID)
  FOR users, edge2 IN INBOUND attr hasLastName, hasFirstName, hasEmail, hasIPAddr, hasPhone, hasDevice

      FILTER users._id != 'Users/36'
      LET e2 = edge2
      COLLECT userBs=users._id INTO g KEEP e2, p

      LET intersect_size = LENGTH(g[*].e2._to)

        // jaccard (setA, setB) = |AintersectB| / (|setA| + |setB| - |AintersectB|)
      LET jaccard_index = intersect_size / ( (6+6) - intersect_size)
      SORT jaccard_index DESC

      //FILTER jaccard_index > TO_NUMBER(@threshold)
      //INSERT {_from: userBs, _to: @input_user, similarity: jaccard_index} INTO sameAs
    
      RETURN {
        "users":  userBs,
        "jaccard": jaccard_index,
        "matching attributes": g[*].e2._to,
        p: g[*].p
       }
"""
)
res = [doc for doc in results]
for r in res:
  print(r)


The result is a list of users most similar to ‘Users/36’ in descending order of their computed jaccard values.

Uncommenting lines 17, we can supply a threshold like “0.7” to filter out users with a jaccard value less than 0.7.  Re-running with a @threshold value of “0.7” will return only the top 2 rows from previous result (i.e. Users/307 and Users/309).

##### Figure 3

!["same-as-user"](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/entity_detection_images/sameAs_Users.png?raw=1)

In this example we are using an INSERT to populate the sameAs edge.  In real life, you probably want to do an UPSERT instead.  An UPSERT will insert if the edge is not there, otherwise it will perform an update.

# **Finding the Right Similarity Algorithm**


While Jaccard Similarity might be good enough for certain domains and use-cases it does not differentiate between the types of attributes.  Let’s see how Jaccard is computed.

##### Figure 4

!["jaccard"](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/entity_detection_images/jaccard_math_notation.png?raw=1)


It treats all attributes with equal weight. You can compute Jaccard by knowing the cardinalities of 3 sets.  Jaccard does not differentiate between a last name match versus a match on device IDs.  In our example, we can argue that a match on device ID is actually worth more than a match on last names.

In some cultures, you find very common last names.  For example, the last name “Singh” in India, there are a lot of people with that last name.  Or first name “David”.  On the other hand, a device ID uniquely identifies a specific mobile device.  So we can say a match on device ID should be given a heavier weighting compared to a match on last name.

In the next AQL example, we show a custom similarity algorithm that assigns different weights for different attribute types.
## Example 2 – Find Similar Users using a custom algorithm

In [ ]:
results = database.aql.execute("""
// For each input_user attribute
FOR attr, edge IN 1..1 OUTBOUND 'Users/36' GRAPH 'IDGraph'

       // Get all users with 1 or more shared attributes 
  FOR users, edge2 IN INBOUND attr hasLastName, hasFirstName, hasEmail, hasIPAddr, hasPhone, hasDevice

      FILTER users._id != 'Users/36'

      LET attr_type = SPLIT(edge2._to,'/')[0]  // eg. "LastName/Vigurs" ---> "LastName"

      COLLECT userids=users._id INTO g KEEP attr_type

        // eg. g[0].attr_type = ["DeviceID", "LastName", "Phone", "Email" ]

      LET sum1 =  'DeviceID' IN g[*].attr_type ?  25 : 0
      LET sum2 =  'IPAddr' IN g[*].attr_type ?    sum1+15 : sum1
      LET sum3 =  'LastName' IN g[*].attr_type ?  sum2+10 : sum2
      LET sum4 =  'FirstName' IN g[*].attr_type ? sum3+10 : sum3
      LET sum5 =  'Phone' IN g[*].attr_type ?     sum4+20 : sum4
      LET total = 'Email' IN g[*].attr_type ?     sum5+20 : sum5

      FILTER total > TO_NUMBER(70) // threshold of 70
      SORT total DESC

      RETURN {
        "users":  userids,
        "similarity_score": total,
        "matching attributes": g[*].attr_type
      }
"""
)

for r in results:
  print(r)

In this example, we are doing the same 2-hop traversal pattern as in example 1.  What’s different is we are aggregating on the types of attributes.  We then process the aggregated attribute types to assign different weights for each attribute type.  We assign a weight of 25 for a deviceID match, a weight of 15 for IP Address match, 10 for last name, etc.  Finally, we compute the total sum of the weights for all matched attribute types.  This sum is then used as the similarity value.

Now Users/307 has a higher similarity score than Users/309 because it has a match on DeviceID.  Using Jaccard in example 1, both these users have similar jaccard values.

# **Next Steps**

In this tutorial we’ve learned how to do Entity Resolution in ArangoDB.  We showed examples on how to build a probabilistic graph for your entities.  If you would like to continue learning more about ArangoDB, here are some next steps to get you started!

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLJoin&utm_medium=Github&utm_campaign=ArangoDB%20University)
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* [ArangoDB Training Center](https://www.arangodb.com/learn/)
* [Getting Started with ArangoDB – Udemy](https://www.udemy.com/course/getting-started-with-arangodb/)

## Continue Reading

* [ArangoML Series: Multi-Model Collaboration](https://www.arangodb.com/2021/01/arangoml-series-multi-model-collaboration/)
* [State of the Art Preprocessing and Filtering with ArangoSearch](https://www.arangodb.com/2020/12/state-of-the-art-preprocessing-and-filtering-with-arangosearch/)
* [ArangoML Series: Intro to NetworkX Adapter](https://www.arangodb.com/2020/11/arangoml-series-intro-to-networkx-adapter/)